In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime as dt
import itertools

%matplotlib inline

Reading data from the CSV into a dataframe

In [ ]:
folder='./Datasets/'
raw_data_1 = pd.read_csv(folder+'2000-01.csv')
raw_data_2 = pd.read_csv(folder+'2001-02.csv')
raw_data_3 = pd.read_csv(folder+'2002-03.csv') 
raw_data_4 = pd.read_csv(folder+'2003-04.csv')
raw_data_5 = pd.read_csv(folder+'2004-05.csv')
raw_data_6 = pd.read_csv(folder+'2005-06.csv')
raw_data_7 = pd.read_csv(folder+'2006-07.csv')
raw_data_8 = pd.read_csv(folder+'2007-08.csv')
raw_data_9 = pd.read_csv(folder+'2008-09.csv')
raw_data_10 = pd.read_csv(folder+'2009-10.csv')
raw_data_11 = pd.read_csv(folder+'2010-11.csv')
raw_data_12 = pd.read_csv(folder+'2011-12.csv')
raw_data_13 = pd.read_csv(folder+'2012-13.csv')
raw_data_14 = pd.read_csv(folder+'2013-14.csv')
raw_data_15 = pd.read_csv(folder+'2014-15.csv')
raw_data_16 = pd.read_csv(folder+'2015-16.csv')
raw_data_17 = pd.read_csv(folder+'2016-17.csv')
raw_data_18 = pd.read_csv(folder+'2017-18.csv')
raw_data_19 = pd.read_csv(folder+'2018-19.csv')
raw_data_20 = pd.read_csv(folder+'2019-20.csv')
raw_data_21 = pd.read_csv(folder+'2020-2021.csv')
raw_data_22 = pd.read_csv(folder+'2021-2022.csv')

In [ ]:
print(len(raw_data_19))


In [ ]:
#Gets all the statistics related to the gameplay
columns = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']
playing_stats_1 = raw_data_1[columns]
playing_stats_2 = raw_data_2[columns]
playing_stats_3 = raw_data_3[columns]
playing_stats_4 = raw_data_4[columns]
playing_stats_5 = raw_data_5[columns]
playing_stats_6 = raw_data_6[columns]
playing_stats_7 = raw_data_7[columns]
playing_stats_8 = raw_data_8[columns]
playing_stats_9 = raw_data_9[columns]
playing_stats_10 = raw_data_10[columns]
playing_stats_11 = raw_data_11[columns]
playing_stats_12 = raw_data_12[columns]
playing_stats_13 = raw_data_13[columns]
playing_stats_14 = raw_data_14[columns]
playing_stats_15 = raw_data_15[columns]
playing_stats_16 = raw_data_16[columns]
playing_stats_17 = raw_data_17[columns]
playing_stats_18 = raw_data_18[columns]
#playing_stats_19 = raw_data_19[columns]
playing_stats_20 = raw_data_20[columns]
playing_stats_21 = raw_data_21[columns]
playing_stats_22 = raw_data_22[columns]


In [ ]:
playing_stats_1 

In [ ]:
#Gets the goals agg arranged by teams
def get_goals_agg(playing_stats):
    #Created a dictionary with the teams as keys and the goals scored as values
    teams = {}
    for i in playing_stats.groupby('HomeTeam').mean(numeric_only=True).T.columns:
        teams[i] = []
    for i in range(len(playing_stats)):
        teams[playing_stats['HomeTeam'][i]].append(playing_stats['FTHG'][i])
        teams[playing_stats['AwayTeam'][i]].append(playing_stats['FTAG'][i])
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    GoalsScored = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsScored[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,39):
        GoalsScored[i] = GoalsScored[i] + GoalsScored[i-1]
    return GoalsScored

In [ ]:
get_goals_agg(playing_stats_1)

In [ ]:
def check_teams_with_less_than_38_matches(playing_stats):
    home_counts = playing_stats['HomeTeam'].value_counts()
    away_counts = playing_stats['AwayTeam'].value_counts()
    total_matches = home_counts.add(away_counts, fill_value=0)
    return total_matches[total_matches < 38]
check_teams_with_less_than_38_matches(playing_stats_20)


In [ ]:
def get_goals_conceded(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean(numeric_only=True).T.columns:
        teams[i] = []
    
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        ATGC = playing_stat.iloc[i]['FTHG']
        HTGC = playing_stat.iloc[i]['FTAG']
        teams[playing_stat.iloc[i].HomeTeam].append(HTGC)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGC)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    GoalsConceded = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsConceded[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,39):
        GoalsConceded[i] = GoalsConceded[i] + GoalsConceded[i-1]
    return GoalsConceded

In [ ]:
def get_gss(playing_stat):
    GC = get_goals_conceded(playing_stat)
    GS = get_goals_agg(playing_stat)
   
    j = 0
    HTGS = []
    ATGS = []
    HTGC = []
    ATGC = []

    for i in range(380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HTGS.append(GS.loc[ht][j])
        ATGS.append(GS.loc[at][j])
        HTGC.append(GC.loc[ht][j])
        ATGC.append(GC.loc[at][j])
        
        if ((i + 1)% 10) == 0:
            j = j + 1
    playing_stat = playing_stat.copy()
  
    playing_stat['HTGS'] = HTGS
    playing_stat['ATGS'] = ATGS
    playing_stat['HTGC'] = HTGC
    playing_stat['ATGC'] = ATGC
    
    return playing_stat


In [ ]:
playing_stats_1 = get_gss(playing_stats_1)
playing_stats_2 = get_gss(playing_stats_2)
playing_stats_3 = get_gss(playing_stats_3)
playing_stats_4 = get_gss(playing_stats_4)
playing_stats_5 = get_gss(playing_stats_5)
playing_stats_6 = get_gss(playing_stats_6)
playing_stats_7 = get_gss(playing_stats_7)
playing_stats_8 = get_gss(playing_stats_8)
playing_stats_9 = get_gss(playing_stats_9)
playing_stats_10 = get_gss(playing_stats_10)
playing_stats_11 = get_gss(playing_stats_11)
playing_stats_12 = get_gss(playing_stats_12)
playing_stats_13 = get_gss(playing_stats_13)
playing_stats_14 = get_gss(playing_stats_14)
playing_stats_15 = get_gss(playing_stats_15)
playing_stats_16 = get_gss(playing_stats_16)
playing_stats_17 = get_gss(playing_stats_17)
playing_stats_18 = get_gss(playing_stats_18)
#playing_stats_19 = get_gss(playing_stats_19)
#playing_stats_20 = get_gss(playing_stats_20)
playing_stats_21 = get_gss(playing_stats_21)
playing_stats_22 = get_gss(playing_stats_22)

In [ ]:
playing_stats_1

In [ ]:
def get_points(result):
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0

In [ ]:
def get_cuml_points(matchres):
    matchres_points = matchres.apply(lambda col: col.map(get_points))
    
    for i in range(2, 39):
        matchres_points[i] = matchres_points[i] + matchres_points[i - 1]
    
    matchres_points.insert(loc=0, column=0, value=[0] * matchres.shape[0])
    return matchres_points


In [ ]:
def get_matchres(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean(numeric_only=True).T.columns:
        teams[i] = []

    # the value corresponding to keys is a list containing the match result
    for i in range(len(playing_stat)):
        if playing_stat.iloc[i].FTR == 'H':
            teams[playing_stat.iloc[i].HomeTeam].append('W')
            teams[playing_stat.iloc[i].AwayTeam].append('L')
        elif playing_stat.iloc[i].FTR == 'A':
            teams[playing_stat.iloc[i].AwayTeam].append('W')
            teams[playing_stat.iloc[i].HomeTeam].append('L')
        else:
            teams[playing_stat.iloc[i].AwayTeam].append('D')
            teams[playing_stat.iloc[i].HomeTeam].append('D')
            
    return pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T

In [ ]:
matchres = get_matchres(playing_stats_1)
cum_pts = get_cuml_points(matchres)
cum_pts

In [ ]:
get_matchres(playing_stats_1)

In [ ]:
def get_agg_points(playing_stat):
    matchres = get_matchres(playing_stat)
    cum_pts = get_cuml_points(matchres)
    HTP = []
    ATP = []
    j = 0
    for i in range(380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HTP.append(cum_pts.loc[ht][j])
        ATP.append(cum_pts.loc[at][j])

        if ((i + 1)% 10) == 0:
            j = j + 1
    playing_stat = playing_stat.copy()
    playing_stat['HTP'] = HTP
    playing_stat['ATP'] = ATP
    return playing_stat

In [ ]:
playing_stats_1 = get_agg_points(playing_stats_1)
playing_stats_2 = get_agg_points(playing_stats_2)
playing_stats_3 = get_agg_points(playing_stats_3)
playing_stats_4 = get_agg_points(playing_stats_4)
playing_stats_5 = get_agg_points(playing_stats_5)
playing_stats_6 = get_agg_points(playing_stats_6)
playing_stats_7 = get_agg_points(playing_stats_7)
playing_stats_8 = get_agg_points(playing_stats_8)
playing_stats_9 = get_agg_points(playing_stats_9)
playing_stats_10 = get_agg_points(playing_stats_10)
playing_stats_11 = get_agg_points(playing_stats_11)
playing_stats_12 = get_agg_points(playing_stats_12)
playing_stats_13 = get_agg_points(playing_stats_13)
playing_stats_14 = get_agg_points(playing_stats_14)
playing_stats_15 = get_agg_points(playing_stats_15)
playing_stats_16 = get_agg_points(playing_stats_16)
playing_stats_17 = get_agg_points(playing_stats_17)
playing_stats_18 = get_agg_points(playing_stats_18)
playing_stats_21 =get_agg_points(playing_stats_21)
playing_stats_22= get_agg_points(playing_stats_22)

In [ ]:
playing_stats_1

The function creates a form summary (like "WWDLD") for each team by aggregating the results over the last num games before each matchday.

In [ ]:
#`playing_stat`: a DataFrame containing football match stats
# `num`: how many previous matchdays to include in the form
def get_form(playing_stat, num):
    form = get_matchres(playing_stat)
    form_final = form.copy()

    for i in range(num, 39):
        form_final[i] = ['' for _ in range(len(form))]
        for j in range(num):
            form_final[i] = form_final[i] + form[i - j]

    return form_final


In [ ]:
get_form(playing_stats_1,5)

This function: Adds two new columns to playing_stat:

HMn: home team’s form on matchday n (most recent result from last num games)

AMn: away team’s form on matchday n (same)

In [ ]:
def add_form(playing_stat, num):
    # Get the form string (e.g., "WWL") for each team
    form = get_form(playing_stat, num)
    
    # Initialize lists to store form result for each match (default 'M' for missing)
    num_matches_to_skip = num * 10  # because each matchday has 10 matches
    home_form = ['M'] * num_matches_to_skip
    away_form = ['M'] * num_matches_to_skip
    
    # Start from matchday `num` since earlier form isn't available
    matchday_index = num

    # Loop over all matches starting from where we have enough past data
    for i in range(num_matches_to_skip, 380):
        home_team = playing_stat.iloc[i]['HomeTeam']
        away_team = playing_stat.iloc[i]['AwayTeam']

        # Get the recent form string for both teams
        home_team_form = form.loc[home_team][matchday_index]
        away_team_form = form.loc[away_team][matchday_index]

        # Take the most recent result (first character in form string)
        home_form.append(home_team_form[0])
        away_form.append(away_team_form[0])

        # Move to the next matchday after every 10 matches
        if (i + 1) % 10 == 0:
            matchday_index += 1

    # Add new columns to the DataFrame
    playing_stat['HM' + str(num)] = home_form
    playing_stat['AM' + str(num)] = away_form

    return playing_stat


In [ ]:
add_form(playing_stats_1,1)

In [ ]:
def add_form_df(playing_statistics):
    # Add recent form-based features for 1 to 5 previous matches
    for num in range(1, 6): 
        playing_statistics = add_form(playing_statistics, num)
    return playing_statistics


In [ ]:
playing_stats_1 = add_form_df(playing_stats_1)
playing_stats_2 = add_form_df(playing_stats_2)
playing_stats_3 = add_form_df(playing_stats_3)
playing_stats_4 = add_form_df(playing_stats_4)
playing_stats_5 = add_form_df(playing_stats_5)
playing_stats_6 = add_form_df(playing_stats_6)
playing_stats_7 = add_form_df(playing_stats_7)
playing_stats_8 = add_form_df(playing_stats_8)
playing_stats_9 = add_form_df(playing_stats_9)
playing_stats_10 = add_form_df(playing_stats_10)
playing_stats_11 = add_form_df(playing_stats_11)
playing_stats_12 = add_form_df(playing_stats_12)
playing_stats_13 = add_form_df(playing_stats_13)
playing_stats_14 = add_form_df(playing_stats_14)
playing_stats_15 = add_form_df(playing_stats_15)
playing_stats_16 = add_form_df(playing_stats_16)
playing_stats_17 = add_form_df(playing_stats_17)
playing_stats_18 = add_form_df(playing_stats_18)
playing_stats_21 = add_form_df(playing_stats_21)
playing_stats_22 = add_form_df(playing_stats_22)


In [ ]:
playing_stats_1

In [ ]:
def get_mw(playing_stat):
    j = 1
    MatchWeek = []
    for i in range(380):
        MatchWeek.append(j)
        if ((i + 1)% 10) == 0:
            j = j + 1
    playing_stat['MW'] = MatchWeek
    return playing_stat

In [ ]:
get_mw(playing_stats_1)

In [ ]:
cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3',
        'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5' ]
playing_stats_1 = playing_stats_1[cols]
playing_stats_1